In [1]:
import json
import pandas as pd

In [2]:
inpath = '/Users/dongmeichen/OneDrive - lanecouncilofgovernments/VE-RSPM'

In [3]:
scen = pd.read_csv(f'{inpath}/sensitivity_tests/scenario_list.csv')

In [4]:
scen.head()

,B,C,D,E,F,G,I,P,T,V,S
0,0,1,1,1,0,1,1,0,1,1,B0C1D1E1F0G1I1P0T1V1
1,1,1,1,1,0,1,1,0,1,1,B1C1D1E1F0G1I1P0T1V1
2,2,1,1,1,0,1,1,0,1,1,B2C1D1E1F0G1I1P0T1V1
3,0,2,1,1,0,1,1,0,1,1,B0C2D1E1F0G1I1P0T1V1
4,1,2,1,1,0,1,1,0,1,1,B1C2D1E1F0G1I1P0T1V1


In [5]:
meas = pd.read_csv(f'{inpath}/output/Measures_Sensitivity_Output_2.csv')
meas.head()

,Measure,Units,Description,Value2010,B0C1D1E1F0G1I1P0T1V1,B1C1D1E1F0G1I1P0T1V1,B2C1D1E1F0G1I1P0T1V1,B0C2D1E1F0G1I1P0T1V1,B1C2D1E1F0G1I1P0T1V1
0,HhTotDailyOthParkingCost,USD per day,Total daily non-work parking expenditures by h...,28241.275835,10480.754212,10480.754212,10480.754212,10480.754212,10480.754212
1,HhTotDailyWkrParkingCost,USD per day,Total daily work parking expenditures by house...,37225.367097,11550.788950,11550.788950,11550.788950,11550.788950,11550.788950
2,HhVehTravCostShare,Proportion,Household vehicle Own & out-of-pocket costs sh...,0.171750,0.044480,0.044178,0.043809,0.044701,0.044397
3,HhVehTravCostShareLowInc,Proportion,Low income (0to20K 2010$) household vehicle Ow...,0.312967,0.337755,0.337755,0.337754,0.341369,0.341369
4,MareaCarSvcAutoDvmtProp,Proportion,Average proportion car service vehicle DVMT in...,1.000000,0.983049,0.983047,0.983043,0.981926,0.981928


In [6]:
for x in scen.iterrows():
    print(x)
    break

(0, B                       0
C                       1
D                       1
E                       1
F                       0
G                       1
I                       1
P                       0
T                       1
V                       1
S    B0C1D1E1F0G1I1P0T1V1
Name: 0, dtype: object)


In [11]:
def rows_to_dicts(df):
    out = []
    fields = list(df)
    out = [{field:val for field, val in zip(fields, values)} 
           for _, values in df.iterrows()]
    return out

In [26]:
output = rows_to_dicts(scen)

In [27]:
output[0]

{'B': 0,
 'C': 1,
 'D': 1,
 'E': 1,
 'F': 0,
 'G': 1,
 'I': 1,
 'P': 0,
 'T': 1,
 'V': 1,
 'S': 'B0C1D1E1F0G1I1P0T1V1'}

In [16]:
print(len(test))
len(scen.S.unique())

69984


69984

In [17]:
co2_vals = [x for x in meas['Measure'] if 'CO2' in x]
co2_vals

['UrbanBusCO2e',
 'UrbanBusCO2eRate',
 'UrbanComSvcCO2e',
 'UrbanHhCO2e',
 'UrbanHvyTrkAveCO2eRate',
 'UrbanHvyTrkCO2e',
 'UrbanLdvCO2e',
 'UrbanLdvCO2eRate',
 'UrbanRailCO2e',
 'UrbanVanCO2e']

In [18]:
len(co2_vals)

10

In [28]:
def vals_from_scenario(scenario, meas_df, subset=None):
    subset = meas_df.Measure.unique() if subset is None else subset
    try:
        out = {field: meas_df.loc[meas_df.Measure == field, scenario].values[0]
              for field in subset}
        return out
    except KeyError:
        return {}
    except:
        raise

In [29]:
vals_from_scenario('B0C1D1E1F0G1I1P0T1V1', meas, co2_vals)

{'UrbanBusCO2e': 6469.76823895145,
 'UrbanBusCO2eRate': 1231.55839744099,
 'UrbanComSvcCO2e': 34237.365820358,
 'UrbanHhCO2e': 458407.02164538,
 'UrbanHvyTrkAveCO2eRate': 855.711475785546,
 'UrbanHvyTrkCO2e': 633054.431485929,
 'UrbanLdvCO2e': 500437.687112108,
 'UrbanLdvCO2eRate': 0.00016033916729636498,
 'UrbanRailCO2e': 0.0,
 'UrbanVanCO2e': 7793.299646370079}

In [33]:
def append_scenario_vals(outputs, meas_df, subset=None):
    for d in outputs:
        scenario = d['S']
        scenario_vals = vals_from_scenario(scenario, meas_df, subset)
        d.update(scenario_vals)
    return outputs

In [34]:
test = append_scenario_vals(output, meas, co2_vals)

In [35]:
test[0]

{'B': 0,
 'C': 1,
 'D': 1,
 'E': 1,
 'F': 0,
 'G': 1,
 'I': 1,
 'P': 0,
 'T': 1,
 'V': 1,
 'S': 'B0C1D1E1F0G1I1P0T1V1',
 'UrbanBusCO2e': 6469.76823895145,
 'UrbanBusCO2eRate': 1231.55839744099,
 'UrbanComSvcCO2e': 34237.365820358,
 'UrbanHhCO2e': 458407.02164538,
 'UrbanHvyTrkAveCO2eRate': 855.711475785546,
 'UrbanHvyTrkCO2e': 633054.431485929,
 'UrbanLdvCO2e': 500437.687112108,
 'UrbanLdvCO2eRate': 0.00016033916729636498,
 'UrbanRailCO2e': 0.0,
 'UrbanVanCO2e': 7793.299646370079}

In [36]:
test[-1]

{'B': 2,
 'C': 3,
 'D': 3,
 'E': 3,
 'F': 3,
 'G': 3,
 'I': 3,
 'P': 3,
 'T': 3,
 'V': 2,
 'S': 'B2C3D3E3F3G3I3P3T3V2'}

In [40]:
with open(f'{inpath}/VEScenarioViewer/test_out.json', 'w') as f:
    json.dump(test, f)

In [38]:
!open ./test_out.json

In [39]:
!rm ./test_out.json